# Comparing algorithms on the Internet dataset
The data is taken from [this source](https://kdd.ics.uci.edu/databases/internet_usage/internet_usage.html).

For details on running the comparison, see [this article](https://github.com/catboost/benchmarks/blob/master/comparison_description.pdf).

In [1]:
import sys

sys.path.append("..")

sys.path

['',
 '/home/yannick/bin/anaconda3/envs/py27/lib/python27.zip',
 '/home/yannick/bin/anaconda3/envs/py27/lib/python2.7',
 '/home/yannick/bin/anaconda3/envs/py27/lib/python2.7/plat-linux2',
 '/home/yannick/bin/anaconda3/envs/py27/lib/python2.7/lib-tk',
 '/home/yannick/bin/anaconda3/envs/py27/lib/python2.7/lib-old',
 '/home/yannick/bin/anaconda3/envs/py27/lib/python2.7/lib-dynload',
 '/home/yannick/bin/anaconda3/envs/py27/lib/python2.7/site-packages',
 '/home/yannick/bin/anaconda3/envs/py27/lib/python2.7/site-packages/Keras-1.0.5-py2.7.egg',
 '/home/yannick/bin/anaconda3/envs/py27/lib/python2.7/site-packages/mxnet-0.7.0-py2.7.egg',
 '/home/yannick/bin/anaconda3/envs/py27/lib/python2.7/site-packages/h2ohyperopt-0.3-py2.7.egg',
 '/home/yannick/bin/anaconda3/envs/py27/lib/python2.7/site-packages/pymongo-2.2-py2.7-linux-x86_64.egg',
 '/home/yannick/bin/anaconda3/envs/py27/lib/python2.7/site-packages/pydot_ng-1.0.1.dev0-py2.7.egg',
 '/home/yannick/bin/anaconda3/envs/py27/lib/python2.7/site-pac

In [2]:
%load_ext autoreload

%autoreload 1

In [3]:
from experiment import Experiment
from xgboost_experiment import XGBExperiment
from lightgbm_experiment import LGBExperiment
from catboost_experiment import CABExperiment

/home/yannick/bin/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Load the dataset and set parameters for the experiment.

In [4]:
learning_task = 'classification'
dataset_path = '../prepare_internet/'
n_estimators = 5000
max_hyperopt_evals = 50

experiment = Experiment(learning_task, train_path=dataset_path + 'train',
                        test_path=dataset_path + 'test', cd_path = dataset_path + 'train.cd')
X_train, y_train, X_test, y_test, cat_cols = experiment.read_data()

# X_train, y_train = X_train[:1000,:], y_train[:1000]

Write a function that does the following:

**Preprocess the dataset:**
calculate counter values on the training set, transform cat features (for the `XGBoost` and `LightGBM` algorithms), and convert the data to the format of the algorithm. Do the same thing with all pairs after splitting them into cross validation folds.

**Select the optimal number of trees for the algorithm with the default parameters:** for each fold and each number of trees, get the result of the algorithm trained on the other four folds, average the results for each number of trees, and choose the best one. 

**Assess the quality of the algorithm with the default parameters on a test dataset:** train the algorithm with the number of trees obtained in the previous step, and calculate the metric value on the test dataset.

**Tune parameters for the algorithm using `Hyperopt`:** on each `Hyperopt` iteration of the algorithm, the best value of trees is selected and the metric for cross validation is calculated.

**Show results on the test dataset for the algorithm with the tuned parameters:** train the algorithm with the optimal parameters and the number of trees obtained in the previous step, and calculate the metric value on the test dataset.

In [5]:
import numpy as np

def run_experiment(Experiment, title):
    experiment = Experiment(learning_task, max_hyperopt_evals=max_hyperopt_evals,
                            n_estimators=n_estimators)
    cv_pairs, (dtrain, dtest) = experiment.split_and_preprocess(X_train.copy(), y_train, 
                                                                X_test.copy(), y_test, 
                                                                cat_cols, n_splits=5)
 
    default_cv_result = experiment.run_cv(cv_pairs)
    experiment.print_result(default_cv_result, 'Default {} result on cv'.format(title))

    print('\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n')

    default_test_losses = []
    for seed in range(5):
        test_result = experiment.run_test(dtrain, dtest, X_test, params=default_cv_result['params'],
                                          n_estimators=default_cv_result['best_n_estimators'], seed=seed)
        default_test_losses.append(test_result['loss'])
        print 'For seed=%d Test\'s %s : %.5f' % (seed, experiment.metric, default_test_losses[-1])
    print '\nTest\'s %s mean: %.5f, Test\'s %s std: %.5f' % (experiment.metric, np.mean(default_test_losses),
                                                             experiment.metric, np.std(default_test_losses))

    print('\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n')
    print('Hyperopt iterations:\n\n')

    tuned_cv_result = experiment.optimize_params(cv_pairs)

    print('\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n')

    experiment.print_result(tuned_cv_result, 'Tuned {} result on cv'.format(title))

    print('\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n')

    tuned_test_losses = []
    for seed in range(5):
        test_result = experiment.run_test(dtrain, dtest, X_test, params=tuned_cv_result['params'],
                                          n_estimators=tuned_cv_result['best_n_estimators'], seed=seed)
        tuned_test_losses.append(test_result['loss'])
        print 'For seed=%d Test\'s %s : %.5f' % (seed, experiment.metric, tuned_test_losses[-1])
    print '\nTest\'s %s mean: %.5f, Test\'s %s std: %.5f' % (experiment.metric, np.mean(tuned_test_losses),
                                                             experiment.metric, np.std(tuned_test_losses))

    return np.mean(default_test_losses), np.mean(tuned_test_losses)

## `XGBoost`

xgb_default_test_result, xgb_tuned_test_result = run_experiment(XGBExperiment, "XGBoost")

## `LightGBM`

%%time

lgb_default_test_result, lgb_tuned_test_result = run_experiment(LGBExperiment, "LightGBM")

## `CatBoost`

In [ ]:
import catboost

catboost.__version__

'0.3.1'

In [ ]:
%%time

cab_default_test_result, cab_tuned_test_result = run_experiment(CABExperiment, "CatBoost")

Default CatBoost result on cv:

logloss = 0.22561958074
best_n_estimators = 1
params = {'rsm': 1.0, 'random_seed': 0, 'verbose': False, 'thread_count': 8, 'learning_rate': 0.03, 'od_type': 'Iter', 'ctr_border_count': 16, 'ctr_description': ['Borders', 'Counter'], 'iterations': 5000, 'od_wait': 40, 'gradient_iterations': 10, 'depth': 6, 'loss_function': 'Logloss', 'fold_len_multiplier': 2, 'kwargs': {}, 'leaf_estimation_method': 'Newton', 'l2_leaf_reg': 3, 'border_count': 128, 'used_ram_limit': 100000000000}

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

For seed=0 Test's logloss : 0.66447
For seed=1 Test's logloss : 0.65603
For seed=2 Test's logloss : 0.66084
For seed=3 Test's logloss : 0.65967
For seed=4 Test's logloss : 0.65520

Test's logloss mean: 0.65924, Test's logloss std: 0.00337

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Hyperopt iterations:


[1/50]	eval_time=29.98 sec	current_logloss=0.220909	min_logloss=0.220909
[2/50]	eval_time=140.28 sec	current_loglos

## Comparing results

The final table with metric values on the test dataset.

import pandas as pd
import numpy as np
from IPython.display import HTML, display
%pylab inline --no-import-all

test_results = np.array([
    (xgb_default_test_result, xgb_tuned_test_result),
    (lgb_default_test_result, lgb_tuned_test_result),
    (cab_default_test_result, cab_tuned_test_result)
])

diff = 100 * test_results / test_results[2,1] - 100

res = [['{:.6f} ({:+.2f}%)'.format(test_results[i, j], diff[i, j]) for j in range(2)] for i in range(3)]

display(HTML(pd.DataFrame(res, columns=['default', 'tuned'], index=['xgboost', 'lightgbm', 'catboost']).to_html()))

results = [
    ('Tuned CatBoost',   cab_tuned_test_result),
    ('Default CatBoost', cab_default_test_result),
    ('Tuned XGBoost',    xgb_tuned_test_result),
    ('Default XGBoost',  xgb_default_test_result),
    ('Tuned LightGBM',   lgb_tuned_test_result),
    ('Default LightGBM', lgb_default_test_result),
]
names = ['%s\n%.5f' % (name, loss) for name, loss in results]

plt.figure(figsize=(20, 7))
plt.scatter(range(6), zip(*results)[1], s=150)
plt.xticks(range(6), names, fontsize=15)
plt.yticks(fontsize=12)
plt.title('Comparison', fontsize=20)
plt.ylabel(experiment.metric, fontsize=16);